# UBER Big Data Pipeline  
## Pramodini Karwande | Chung-Pang Hsu | Autumn Biggie  

Data is the new Oil! For any company to succeed in today’s world, data is the key. Businesses have grown bigger and systems that drive them have become more complex. There is a vast amount of data that gets generated everyday. In this scenario Data reliability, scalability, observability becomes a very important aspect for every organization. Because without this, businesses cannot leverage their data effectively to make the right decisions.
Before directly jumping to Uber BigData architecture, let’s learn a little bit about Uber first!  

*“We are Uber. The go-getters. The kind of people who are relentless about our mission to help people go anywhere and get anything. Movement is what we do. It’s our lifeblood. It runs through our veins. It’s what gets us out of bed each morning. It pushes us to constantly reimagine how we can move better. For you. For all the places you want to go. For all the things you want to get. For all the ways you want to earn. Across the entire world. In real time. At the incredible speed of  now.”*  

Yes, this is the mission statement of Uber!  In 2009, Uber was founded as Ubercab by Garrett Camp, a computer programmer. After Camp and his friends spent $800 hiring a private driver, he wanted to find a way to reduce the cost of direct transportation. He realized that sharing the cost with people could make it affordable, and his idea morphed into Uber.  

Uber is committed to delivering safer and more reliable transportation across the global markets. Every day in over 10,000 cities around the world, millions of people rely on Uber to travel, order food, and ship cargo. Uber apps and services are available in over 69 countries and run 24 hours a day. One magical click and car shows up, one magical click and food shows up and behind the scenes, data is powering a lot of experiences.  

At a global scale, these activities generate large amounts of logging & operational data that runs through Uber’s systems in real-time. This includes information about consumer demand, driver-partner availability, and other operational tasks such as payments, notifications, and more. Operating a complex marketplace requires personnel working at Uber like engineers, data scientists, data analysts, and operations managers to take real-time business decisions based on trends observed on Uber platform.  To accomplish this, Uber relies heavily on making data-driven decisions at every level, from forecasting rider demand during high traffic events to identifying and addressing bottlenecks in driver-partner sign-up process. Over time, the need for more insights has resulted in over 100 petabytes of analytical data that needs to be cleaned, stored, and served with minimum latency through Apache Hadoop® based Big Data platform. Since 2014, data engineers have worked to develop a Big Data solution that ensures data reliability, scalability, and ease-of-use.  

Now, let’s deep dive into Uber’s Data Journey using BigData Pipeline.  Their initial solution, developed before 2014, was based on MySQL and PostgreSQL. The relatively small amount of data Uber had then - a few TB - could fit into these RDBMSs, and users had to figure out themselves how to query across databases. The city operations teams, data scientists and analysts, and engineering teams used this data. To manage data volume, variety, growth and in terms of numbers shown in Table[1], Uber built a foundation of big data systems (please refer Fig[1]) by embracing standard technology of systems including Spark, Hive, Presto, HDFS, YARN, and Parquet.  


Scale of Big Data Platform:  

| Source    |   Quantity   |
| ------------- | --------------- |
| Hosts | ~ 20 K               |
| HDFS Data | > 100 PB         |
| VCores | > 200 K             |
| Kafka Hosts | ~ 2 K          |
| Presto Hive queries/day | > 100 K |
| Spark jobs/day | > 100 K |
| Messages Processed/day | > 2 Trillion |

Table [1]  



![](anytoanyimage.png)  
Figure [1]  

## Main purpose of most of the major software in their big data pipeline  

In Generation 1 (before 2014) of Uber’s Big Data Platform, the company used MySQL and PostgreSQL to store their data, which was only a few terabytes at the time. Uber engineers could write their own code to combine data from different tables, and the data was scattered across a few different traditional online transaction processing (OLTP) databases. As both the company and the amount of data grew, they decided to use __Vertica__ as their _data warehouse_ platform in order to collect all of their data into one accessible place for querying. Vertica is fast, scalable, and column oriented, making quick data queries and modeling easier. They also standardized SQL for their platform to streamline data access and built an online query service for their users.  

Due to data reliability, storage costs, and scalability issues, Generation 2 arrived with the usage of new software. Uber began using a __Hadoop__ _data lake_ to gather data from different data stores once, without any data transformations during the process. This resulted in increased data reliability, less superfluous data copies, scalable ingestion, and significantly reduced the pressure on Generation 1 data stores. To facilitate user access to the new Hadoop system, Uber began using Presto, Apache Spark, and Apache Hive. __Presto__ is used for interactive data queries as needed, while __Apache Spark__ is used for access to raw data using programmatic (SQL and non-SQL) formats. To facilitate massive queries, __Apache Hive__ is used. In addition, the company switched from JSON to the Apache Parquet file format, which integrated well with __Spark__ and saved on storage space due to its columnar structure, making it perfect for analytic queries.   

Due to inefficiency of ingestion jobs and data latency, Generation 3 of Uber’s Big Data Platform came out in 2017. The major addition was the Hadoop Upserts anD Incremental (__Hudi__), an open source Spark library that supports data updates and deletions, as well as the ability to operate on an incremental ingestion model. That is, users are able to pass a timestamp and retrieve only updated records, significantly reducing data latency from 24 hours to less than one hour. Consequently, Hadoop raw data tables using Hudi storage are provided in two viewing modes: Latest Mode View and Incremental Mode View. While Latest Mode View fetches the entire current table, Incremental Mode View fetches only the updates since the provided timestamp (see figure 2 below). Other software updates from Generation 3 include the addition of __Apache Kafka__, which formalizes the transfer of datastore changes between the storage and big data teams. Upstream datastore events are sent to Kafka with attached metadata for further processing by both teams. __Marmaray__ is also used as the data ingestion platform, which runs in mini-batches and applies the datastore changes from Kafka (with its metadata) on top of the existing data in Hadoop. Since Ingestion Spark jobs occur every 10-15 minutes, data latency has been reduced to 30 minutes.  

![](ingestion.png)  
Figure 2

## Types of problems Uber faces with their data  

We summarize our ongoing efforts to enhance Uber’s Big Data platform for **improved data quality, data latency, efficiency, scalability and reliability.**  

_Data quality:_  
There are two issues. The one is non-schema-conforming data. When some of the upstream data, stores do not mandatorily enforce or check data schema before storage the quality of the actual data content. The other is the quality of the actual data content. While using schemas ensures that data contains correct data types, they do not check the actual data values.  

_Data latency:_ 
There is long raw data latency in Hadoop and for modeled tables. After reducing those data latencies, this will allow more use cases to move away from stream processing to more efficient mini-batch processing.  

_Data efficiency:_  
They are relying on dedicated hardware for any of our services and little service dockerization. In addition, they were not unifying all of our resource schedulers within and across our Hadoop ecosystem to bridge the gap between our Hadoop and non-data services across the company.  

_Data scalability and reliability:_  
While the ingestion platform was developed as a generic, pluggable model, the actual ingestion of upstream data still includes a lot of source-dependent pipeline configurations, making the ingestion pipeline fragile and increasing the maintenance costs of operating several thousands of these pipelines.  

Finally, Hudi will potential provide the solution for problems above. This significantly increases the write amplification, especially when the ratio of update to insert increases, and prevents creation of larger Parquet files in HDFs. The new version of Hudi is designed to overcome this limitation by storing the updated record in a separate delta file and asynchronously merging it with the base Parquet file based on a given policy. Having Hadoop data stored in larger Parquet files as well as a more reliable source-independent data ingestion platform will allow analytical data platform to continue grow in the upcoming years as the business thrive.


__Summary:__
Based on what we have learned in our course work, we can all apply here and observe how Big Data Concepts gets used in real world. Big Data is data that we can't handle normally i.e. Data don't fit at single computer and/or Data is constantly being updated/added. As per above Uber's business model, we can say that Uber's data is very large that can't handle normally. As well as, data is constantly being updated/added. 

We learned about Big Data Platform common concepts `5 V's` and we can relate how that has been worked in Uber's Big Data Pipeline. Please refer below Figure 3.

![](UBerBigDataV5s.png)  
Figure 3



<br />We have seen standard Big Data Pipeline(Figure 4) which has been modified a lot by Uber to resolve the issues they faced as per their business requirements and get mature the Big data Pipeline system in real World. 

![](standardBigData.PNG)  
Figure 4



<br />__Uber's Big Data Pipeline Platform project made us understand how Big Data getting used in real world application and how this technology helps. It saves time, cost, energy and provide better life to people.__


<br />__References:__

https://eng.uber.com/uber-big-data-platform/  
https://eng.uber.com/metadata-insights-databook/  
https://eng.uber.com/improving-hdfs-i-o-utilization-for-efficiency/  
https://www.youtube.com/watch?v=aveES6gn1Gs  
https://eng.uber.com/kafka-async-queuing-with-consumer-proxy/  
